# Clean Dialogue Demo

In [1]:
from queue import Queue
from rtai.utils.config import YamlLoader
from rtai.llm.llm_client import LLMClient
from rtai.agent.agent_manager import AgentManager
from rtai.agent.agent import Agent
from rtai.world.world import World
from rtai.world.clock import clock

cfg = YamlLoader.load("configs/rtai_neil.yaml")
timer = clock(cfg.expand('Clock'))
event_queue = Queue()

world = World(cfg.expand('World'), event_queue)

llm_client = LLMClient()
llm_client.initialize(cfg.expand('LLMClient'))

agent_mgr = AgentManager(event_queue, cfg.expand('Agents'), client=llm_client, world=world)
agent_mgr.initialize()

mark: Agent = agent_mgr.agents['Mark Turner']
sarah: Agent = agent_mgr.agents['Sarah Reynolds']
emily: Agent = agent_mgr.agents['Emily Thorton']

/opt/homebrew/Caskroom/miniforge/base/envs/embeddings_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### First, we create the facts of the case

In [3]:
sarah_facts = ["Sarah witnessed a heated argument between Emily and Richard Thornton during the social gathering.",
"She noticed Emily and Richard's strained relationship in the weeks leading up to the event.",
"Sarah is aware of financial difficulties in the Thornton marriage, with Richard contemplating a divorce.",
"During the argument, Sarah overheard Emily expressing frustration about Richard's intention to file for divorce.",
"Sarah is acquainted with other guests at the event who had their own reasons to dislike Richard Thornton."]

mark_facts = [
    "Richard had business rivals who might have had motives to harm him.",
    "There is a potential alibi for Emily during the time of the poisoning, involving a witness not yet presented in court.",
    "Inconsistencies exist in the initial police investigation, including gaps in the timeline and overlooked potential suspects.",
    "Richard was involved in a secret business deal that might have contributed to his strained relationships.",
    "Mark possesses an anonymous letter received by the Thorntons, hinting at threats from an unknown source."
]

emily_facts = [
    "Emily is aware of the severe financial struggles within her marriage to Richard.",
    "They had heated arguments about money, and Richard was considering a divorce partly due to their financial difficulties.",
    "Emily knows about the strained relationship between her and Richard leading up to the night of the incident.",
    "The tension was exacerbated by discussions about divorce, and Emily felt the emotional weight of their crumbling marriage.",
    "Emily received anonymous threatening letters in the weeks leading up to the incident.",
    "The letters hinted at harm coming to Richard and warned her about potential dangers.",
    "Emily kept this information to herself, unsure of the credibility of the threats.",
    "Emily was aware of Richard's involvement in a secret business deal that had caused friction in their personal and professional lives.",
    "The details of this deal were not fully disclosed to her, but she knew it played a role in their marriage difficulties.",
    "Emily can provide insight into her emotional state on the night of the incident.",
    "She may reveal her emotional turmoil, fears, and frustrations leading up to the events.",
    "Emily recalls an unknown person visiting their home on the night of the incident.",
    "The identity and purpose of this person remain a mystery to her."
]
# add the facts to agents' memories
def add_facts(agent, facts):
    for fact in facts:
        agent.l_mem.add_concept(fact)
add_facts(sarah, sarah_facts)
add_facts(mark, mark_facts)
add_facts(emily, emily_facts)

### Initialize the LLM

In [93]:
import guidance
from guidance import models
from guidance import gen, select

local_model_path = cfg.expand("LLMClient").get_value("local_model_path")
mistral = models.LlamaCpp(local_model_path, n_gpu_layers=-1, n_ctx=7000)

# initialize mythos as a scriptwriter
mythos = mistral + "You are a master Hollywood scriptwriter who specializes in legal dramas. Conduct each task as if you are that scriptwriter."

### Generate the dialogue

In [6]:
dialogue_topic = "Mark is helping Sarah prepare for taking the stand"
location = "Mark's office"

In [49]:
@guidance
def generate_dialogue(lm, agent1, agent2, location, dialogue_topic):
    agent1_name = agent1.get_name()
    agent2_name = agent2.get_name()
    agent1_persona = agent1.get_common_set_str()
    agent2_persona = agent2.get_common_set_str()
    agent1_ctx = agent1.l_mem.retriever.retrieve_context(dialogue_topic)
    agent2_ctx = agent2.l_mem.retriever.retrieve_context(dialogue_topic)
    prompt = f'''
    Generate a short dialogue between {agent1_name} and {agent2_name} in {location} about {dialogue_topic}

    {agent2_name}'s persona: {agent2_persona}
    {agent2_name}'s context: {agent2_ctx}
    Mark's persona: {agent1_persona}
    Mark's context: {agent1_ctx}

    Here is the short dialogue:
    {gen('dialogue', temperature=1.0, max_tokens=1000)}"""
    '''
    lm += prompt
    return lm

# mythos += generate_dialogue(mark, sarah, location, dialogue_topic)

In [14]:
# access the generated dialogue and reset the state of storyteller model
dialogue = mythos["dialogue"]
mythos.reset()

Now, let's export the dialogue for visualization

In [15]:
def clean_dialogue(dialogue):
    cleaned = dialogue.split("\n")
    speakers = []
    utterances = []
    for line in cleaned:
        if line == '':
            continue
        else:
            print(line)
            split_line = line.split(":", maxsplit=1)
            try:
                speaker, utterance = split_line[0], split_line[1]
                speakers.append(speaker)
                utterances.append(utterance)
            except:
                continue
    return speakers, utterances

In [16]:
import pandas as pd

speakers, utterances = clean_dialogue(dialogue)
df = pd.DataFrame({'speaker': speakers, 'utterance': utterances})
df

Mark Turner: (sitting behind the desk, poring over files) Sarah, thank you for coming by today. I wanted to discuss some details regarding your testimony.
Sarah Reynolds: (entering the office, visibly anxious) Yes, Mark, I understand that this is a critical moment for us. I will do whatever it takes to help Emily.
Mark Turner: (looking up from the files, in a calming tone) I know that, Sarah, but it's also important that the truth come out. We cannot simply sweep anything under the rug. I need to understand exactly what you saw and heard during the party.
Sarah Reynolds: (hesitating) Well, I heard Emily and Richard arguing about the impending divorce. I also saw a few guests looking rather heated during the exchange, but I couldn't place who they were.
Mark Turner: Could you be more specific about who those guests were? Or even their general description? Every detail, however minor, might help us in the long run.
Sarah Reynolds: (nervously) I'm really not sure, Mark. It was dark, and t

,speaker,utterance
0,Mark Turner,"(sitting behind the desk, poring over files) ..."
1,Sarah Reynolds,"(entering the office, visibly anxious) Yes, M..."
2,Mark Turner,"(looking up from the files, in a calming tone..."
3,Sarah Reynolds,"(hesitating) Well, I heard Emily and Richard ..."
4,Mark Turner,Could you be more specific about who those gu...
5,Sarah Reynolds,"(nervously) I'm really not sure, Mark. It was..."
6,Mark Turner,"(writing down notes, pensive) Very well. Now,..."
7,Sarah Reynolds,(shaking her head) I'm afraid I cannot divulg...
8,Mark Turner,"(examining her intently) Sarah, your honesty ..."
9,Sarah Reynolds,"(nodding) I will do that, Mark. Thank you for..."


In [17]:
df.to_csv('dialogues/dialogue5.csv', index=False)

In [10]:
mark.l_mem.retriever.retrieve_context(dialogue_topic)

recency score for There is a potential alibi for Emily during the time of the poisoning, involving a witness not yet presented in court. is 10.0
recency score for Mark possesses an anonymous letter received by the Thorntons, hinting at threats from an unknown source. is 10.0
recency score for Richard was involved in a secret business deal that might have contributed to his strained relationships. is 10.0
recency score for Richard had business rivals who might have had motives to harm him. is 10.0
recency score for Inconsistencies exist in the initial police investigation, including gaps in the timeline and overlooked potential suspects. is 10.0


'There is a potential alibi for Emily during the time of the poisoning, involving a witness not yet presented in court.Mark possesses an anonymous letter received by the Thorntons, hinting at threats from an unknown source.Richard was involved in a secret business deal that might have contributed to his strained relationships.'

### Reflect on the dialogue

In [56]:
from guidance import select

@guidance
def reflect(lm, agent1, agent2, dialogue):
    agent1_name = agent1.get_name()
    agent2_name = agent2.get_name()
    agent1_ctx = agent1.l_mem.retriever.retrieve_context(dialogue)

    lm += f'''Before chattting with {agent2_name}, {agent1_name} knew {agent1_ctx}.

    Given this dialogue, what are the top 3 NEW takeaways for {agent1_name} from this dialogue?

    Here is the dialogue: {dialogue}

    Here are the takeaways:
    {gen(name="takeaways", temperature=0.8, max_tokens=300)}'''

    lm += f'''
    Based off those three takwaways what should {agent1_name} do next?

    {agent1_name}'s options are to THEORIZE: take time to theorize a timeline about what happened, CHAT: go chat with another witness to gather more information
    {select(name="do_next", options=["THEORIZE", "CHAT"])} because {gen(name="reason", temperature=0.8, max_tokens=300)}'''
    if lm["do_next"] == "THEORIZE":
        lm += f'''Q: What is {agent1_name}'s current theory about what happened?
        A: {gen(name="theory", temperature=0.8, max_tokens=300)}.'''
    if lm["do_next"] == "CHAT":
        lm += f'''Q: Who should {agent1_name} chat with next to learn more information about the case? Prioritize witnesses who have not been interviewed yet.
        A: {select(name="next_chat", options=["Emily Thorton", "Emily Thorton"])}. The one liner topic of the chat will be: {gen(name="topic", stop=".", temperature=0.8, max_tokens=100)}.'''
    return lm

# if lm["do_next"] == "CHAT":
#     lm += f'''Q: Who should {name} chat with next?
#     A: {select(name="next_chat", options=["Emily Thorton", "Sarah"])}. Why? {gen(name="reason", temperature=0.8, max_tokens=300)}.'''

# @guidance
# def act(lm, name, options):
#     lm += f'''What should {name} do next?
#     {select(name="do_next", options=options)}'''
#     return lm
prompt = f'''Who should Mark chat with next? {select(name="next_chat", options=["Emily Thorton", "Sarah"])}'''

mythos_reasoning = mistral + "You are analyzing what Mark Turner learned from a chat with Sarah."
mythos_reasoning += reflect(agent1=mark, agent2=sarah, dialogue=dialogue)
mythos_reasoning += prompt

In [57]:
# Now we add the takeaways from the dialogue to Mark's memory
takeaways = mythos_reasoning["takeaways"].split("\n")

# clean the takeaways
for takeaway in takeaways:
    if takeaway == '':
        continue
    else:
        cleaned_takeaway = takeaway.split(".", maxsplit=1)[1]
        print(cleaned_takeaway)
        mark.l_mem.add_concept(cleaned_takeaway)

 Emily admitted to leaving the room after the heated argument with Richard, providing a potential gap in time for the poisoning to have occurred.
 Sarah's testimony about the man in a red tie provides a new suspect or potential lead in the investigation.
 Emily's financial struggles and Richard's infidelity may have been motives for the poisoning, but they do not definitively prove Emily's guilt. These details add complexity to the case and make it more intriguing for Mark to investigate further.


In [58]:
mark.l_mem.retriever.retrieve_context("Emily")

recency score for  Emily admitted to leaving the room after the heated argument with Richard, providing a potential gap in time for the poisoning to have occurred. is 10.0
recency score for There is a potential alibi for Emily during the time of the poisoning, involving a witness not yet presented in court. is 10.0
recency score for  Emily's financial struggles and Richard's infidelity may have been motives for the poisoning, but they do not definitively prove Emily's guilt. These details add complexity to the case and make it more intriguing for Mark to investigate further. is 10.0
recency score for  Sarah's testimony about the argument between Emily and Richard, as well as the heated exchanges from other guests, reinforces the possibility of a motive for the poisoning. Mark now has more details to work with and may be able to identify potential suspects based on this new information. is 10.0
recency score for  Sarah observed a man in a red tie leaving the room shortly after the heate

" Emily admitted to leaving the room after the heated argument with Richard, providing a potential gap in time for the poisoning to have occurred.There is a potential alibi for Emily during the time of the poisoning, involving a witness not yet presented in court. Emily's financial struggles and Richard's infidelity may have been motives for the poisoning, but they do not definitively prove Emily's guilt. These details add complexity to the case and make it more intriguing for Mark to investigate further."

In [45]:
chat_next = mythos_reasoning["next_chat"]
about = mythos_reasoning["topic"]
mythos_reasoning.reset()

'"Emily, I need to discuss the heated exchange between you and Richard during the party'

In [38]:
# do next action
next_action = mythos_reasoning["do_next"]
who = mythos_reasoning["next_chat"]
f"Mark's next action is to {next_action} with {who}"

"Mark's next action is to CHAT with Emily Thorton"

In [47]:
mythos.reset()

In [50]:
mythos += generate_dialogue(mark, emily, location, dialogue_topic=about)

In [51]:
# access the generated dialogue and reset the state of storyteller model
dialogue = mythos["dialogue"]
mythos.reset()

In [54]:
import pandas as pd

speakers, utterances = clean_dialogue(dialogue)
df2 = pd.DataFrame({'speaker': speakers, 'utterance': utterances})
df2.to_csv('dialogues/dialogue6.csv', index=False)

---
Mark Turner: Emily, I need to discuss the heated exchange between you and Richard during the party.
Emily Thorton: (folds her arms, slightly defensive) Mark, I'd rather focus on solving the issue at hand...the fact that my husband is dead.
Mark Turner: I understand that, Emily. However, it's crucial that we consider every possible angle in this investigation. I believe the heated exchange between you and Richard could be relevant to unraveling what happened that night.
Emily Thorton: (sighs) I've told you the truth about our rocky marriage, Mark. Our financial struggles and Richard's infidelity took a toll on us. That night, we argued about the potential divorce and his business dealings. I can't deny that things heated up between us, but I didn't harm him.
Mark Turner: I believe you, Emily. But I need to know all the details regarding that night. When were you last in contact with Richard before the incident? What did you both say to each other? Did you leave the room at any point

In [71]:
# Now we summarize
df3 = pd.read_csv('dialogues/dialogue5.csv')
df4 = pd.read_csv('dialogues/dialogue6.csv')

In [78]:
def string_rep(df):
    concatenated_string = ', '.join(df3.apply(lambda row: f"{row['speaker']}:{row['utterance']}", axis=1))
    return concatenated_string

In [94]:
prompt = f'''

Here are character summaries for Mark, Emily, and Sarah:
Mark Character Summary: {mark.get_common_set_str()}
Emily Character Summary: {emily.get_common_set_str()}
Sarah Character Summary: {sarah.get_common_set_str()}

Here is the first conversation between Mark and Sarah:
{string_rep(df3)}

Here is the second conversation between Mark and Emily:
{string_rep(df4)}

Now, create a detective fiction vignette from Mark's persepctive (SPEAK IN THIRD PERSON) for a young adult audience that incorporates each character's personality and quotes from the two conversations between Mark and Sarah and Mark and Emily. Make it dramatic and interesting to read. Keep each line SHORT AND SNAPPY. Incorporate metaphors and symbolism.
{gen(name="story", temperature=1.0, max_tokens=1000)}
'''

In [95]:
fiction = mistral + "You are a seasoned detective fiction writer. You incorporate inspiration tastefully and in an entertaining way."
mythos += prompt

In [101]:
story_line = []
mythos["story"].split("\n")
for line in mythos["story"].split("\n"):
    if line == '':
        continue
    else:
        story_line.append(line)
story_df = pd.DataFrame(story_line, columns=['story'])
story_df.to_csv('dialogues/story.csv', index=False)